# Video 2 - Group Chat

## Problem Statement
1. **New Project Focus**: A mobile gaming company is developing a game for PS5, requiring new expertise.

2. **HR Role**: The HR team is responsible for determining hiring requirements by collaborating with relevant departments.

3. **Agentic System**: Implement a group chat system where HR initiates discussions with a Group Chat Manager to gather hiring needs from team members.


In [1]:
# Import the libraries
import autogen
from autogen import ConversableAgent, GroupChat, GroupChatManager
from IPython.display import Markdown, display

In [2]:
# load env variables
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# configuration for LLM
config_list = {
    "config_list": [{"model": "gpt-4o-mini", "temperature": 0.1}]
}

In [4]:
hr_agent = ConversableAgent(
    "hr_agent",
    system_message="""You are the head of HR in a company. The company is initiating the development of 
                    a new B2C tech product and aims to assemble a dedicated team for this project. 
                    Gather the hiring requirements from different departments as needed.""",
    llm_config=config_list,
    human_input_mode="NEVER",  
)

In [5]:
product_agent = ConversableAgent(
    "product_agent",
    system_message="""You are the Head of Product at a company developing a new B2C tech product. 
        Your goal is to communicate with the HR department to outline the hiring requirements for assembling a 
        dedicated Product management team for this project. Clearly and concisely list the key roles, qualifications, and 
        skills, and budget needed for this initiative.""",
    llm_config=config_list,
    human_input_mode="NEVER",  
)

In [6]:
engineering_agent = ConversableAgent(
    "engineering_agent",
    system_message="""You are the Head of Engineering at a company developing a new B2C tech product. 
        Your goal is to communicate with the HR department to outline the hiring requirements for assembling a 
        dedicated engineering team for this project. Clearly and concisely list the key roles, qualifications, and 
        skills, and budget needed for this initiative.""",
    llm_config=config_list,
    human_input_mode="NEVER",  
)

In [7]:
marketing_agent = ConversableAgent(
    "marketing_agent",
    system_message="""You are the Head of Marketing at a company developing a new B2C tech product. 
        Your goal is to communicate with the HR department to outline the hiring requirements for assembling a 
        dedicated marketing team for this project. Clearly and concisely list the key roles, qualifications, and 
        skills, and budget needed for this initiative.""",
    llm_config=config_list,
    human_input_mode="NEVER",  
)

In [8]:
customer_support_agent = ConversableAgent(
    "customer_support_agent",
    system_message="""You are the Head of Customer support at a company developing a new B2C tech product. 
        Your goal is to communicate with the HR department to outline the hiring requirements for assembling a 
        dedicated customer support team for this project. Clearly and concisely list the key roles, qualifications, and 
        skills, and budget needed for this initiative.""",
    llm_config=config_list,
    human_input_mode="NEVER",  
)

- The selection criteria is auto.
- Max-round = 5, means the conversation will go five rounds. 

In [9]:
group_chat = GroupChat(
    agents=[customer_support_agent, product_agent, engineering_agent, marketing_agent],
    messages=[],
    max_round= 5,
    speaker_selection_method='auto',
    select_speaker_auto_llm_config=config_list
)

In [10]:
group_chat_manager = GroupChatManager(
    groupchat=group_chat,
)

In [11]:
chat_result = hr_agent.initiate_chat(
    group_chat_manager,
    message="A gaming company focussed on mobiles games is launching a new game designed for PS4. Provide hiring requirements for this new product",
    summary_method="last_msg",
)

hr_agent (to chat_manager):

A gaming company focussed on mobiles games is launching a new game designed for PS4. Provide hiring requirements for this new product

--------------------------------------------------------------------------------

Next speaker: product_agent

product_agent (to chat_manager):

Subject: Hiring Requirements for Product Management Team for New PS4 Game

Dear HR Team,

As we embark on the development of our new mobile game designed for PS4, it is essential to assemble a dedicated Product Management team that will drive the project from conception to launch. Below are the key roles, qualifications, skills, and budget considerations for this initiative.

### Key Roles and Responsibilities:

1. **Product Manager (2 Positions)**
   - **Qualifications**: 
     - Bachelor’s degree in Business, Marketing, or a related field.
     - 3-5 years of experience in product management, preferably in the gaming industry.
   - **Skills**:
     - Strong understanding of game d

In [12]:
agent_name_to_find = "customer_support_agent"

# Iterate through the dictionary to find the last conversation for the specified agent
for agent, interactions in group_chat_manager.chat_messages.items():
    # Filter interactions by agent name
    filtered_interactions = [i for i in interactions if i.get("name") == agent_name_to_find]
    if filtered_interactions:
        # Print the last conversation's content
        print(filtered_interactions[-1]["content"])
        break
else:
    print("Agent not found or no interactions available.")


Subject: Hiring Requirements for Customer Support Team for New B2C Tech Product

Dear HR Team,

As we prepare for the launch of our new B2C tech product, it is vital to establish a dedicated customer support team that will provide exceptional service and support to our users. Below are the key roles, qualifications, skills, and budget considerations for this initiative.

### Key Roles and Responsibilities:

1. **Customer Support Manager (1 Position)**
   - **Qualifications**: 
     - Bachelor’s degree in Business Administration, Communications, or a related field.
     - 5+ years of experience in customer support or service management, preferably in tech or B2C environments.
   - **Skills**:
     - Strong leadership and team management skills.
     - Excellent communication and interpersonal skills.
     - Ability to analyze customer feedback and implement improvements.

2. **Customer Support Representatives (3 Positions)**
   - **Qualifications**: 
     - High school diploma or equiva

# Video 4 - StateFlow

In [13]:
summarizer_agent = ConversableAgent(
    "summarizer_agent",
    system_message="""Summarize the received hiring requirements of all the departments, mentioning just the positions for each department""",
    llm_config=config_list,
    human_input_mode="NEVER",  
)

- Not accomodating the steps in case an agent throws an error,
- And conditions under which a specific agent will be approached or ignored, etc.

In [14]:
def state_transition(last_speaker, groupchat):
    
    messages = groupchat.messages

    if last_speaker is hr_agent:
        return product_agent
    elif last_speaker is product_agent:
        if 'software' in messages[-1]['content']:
            return engineering_agent
        # If the product is software centric
        else:
            return marketing_agent
        if messages[-1]["content"] == "exitcode: 1":
            return engineering_agent
    elif last_speaker in [marketing_agent, engineering_agent]:
        return customer_support_agent
    elif last_speaker is customer_support_agent:
        return summarizer_agent
    elif last_speaker == summarizer_agent:
        return None

In [15]:
group_chat = GroupChat(
    agents=[product_agent, engineering_agent, marketing_agent, customer_support_agent, summarizer_agent],
    messages=[],
    max_round=10,
    speaker_selection_method=state_transition,
    select_speaker_auto_llm_config=config_list
)

In [16]:
group_chat_manager = GroupChatManager(
    groupchat=group_chat,
)

In [17]:
chat_result = hr_agent.initiate_chat(
    group_chat_manager,
    message="A gaming company focussed on mobiles games is launching a new game designed for PS4. Provide hiring requirements for this new product",
    summary_method="last_msg",
)

hr_agent (to chat_manager):

A gaming company focussed on mobiles games is launching a new game designed for PS4. Provide hiring requirements for this new product

--------------------------------------------------------------------------------

Next speaker: product_agent

product_agent (to chat_manager):

Subject: Hiring Requirements for Product Management Team for New PS4 Game Launch

Dear HR Team,

As we embark on the development of our new mobile game designed for PS4, it is crucial to assemble a dedicated Product Management team to ensure the project’s success. Below are the key roles, qualifications, skills, and budget considerations for this initiative.

### Key Roles and Responsibilities:

1. **Product Manager (2 Positions)**
   - **Qualifications**: 
     - Bachelor’s degree in Business, Marketing, or a related field.
     - 3-5 years of experience in product management, preferably in gaming.
   - **Skills**:
     - Strong understanding of game development processes.
     - E

In [18]:
chat_result = hr_agent.initiate_chat(
    group_chat_manager,
    message="A toothpaste company has decided to focus on a new mouthwash. Provide hiring requirements for this new product",
    summary_method="last_msg",
)

hr_agent (to chat_manager):

A toothpaste company has decided to focus on a new mouthwash. Provide hiring requirements for this new product

--------------------------------------------------------------------------------

Next speaker: product_agent

product_agent (to chat_manager):

Subject: Hiring Requirements for Product Management Team for New Mouthwash Initiative

Dear HR Team,

As we embark on the development of our new mouthwash product, it is essential to assemble a dedicated Product Management team that will drive the project from conception to launch. Below are the key roles, qualifications, skills, and budget considerations for this initiative.

### Key Roles and Responsibilities:

1. **Product Manager**
   - **Qualifications**: Bachelor’s degree in Marketing, Business, or a related field; MBA preferred.
   - **Skills**: Strong project management skills, excellent communication, market research proficiency, and experience in consumer goods.
   - **Responsibilities**: Overse

In [19]:
agent_name_to_find = "summarizer_agent"

# Iterate through the dictionary to find the last conversation for the specified agent
for agent, interactions in group_chat_manager.chat_messages.items():
    # Filter interactions by agent name
    filtered_interactions = [i for i in interactions if i.get("name") == agent_name_to_find]
    if filtered_interactions:
        # Print the last conversation's content
        print(filtered_interactions[-1]["content"])
        break
else:
    print("Agent not found or no interactions available.")


### Summary of Hiring Requirements by Department

#### Product Management Team (New Mouthwash Initiative)
1. Product Manager
2. Market Research Analyst
3. Product Development Specialist
4. Brand Manager
5. Sales Manager

#### Engineering Team (New B2C Tech Product)
1. Software Engineer (Frontend)
2. Software Engineer (Backend)
3. DevOps Engineer
4. Quality Assurance Engineer
5. UX/UI Designer

#### Customer Support Team (New B2C Tech Product)
1. Customer Support Manager
2. Customer Support Representative
3. Technical Support Specialist
4. Customer Experience Analyst
5. Training and Quality Assurance Specialist


# Video 6 - Nested Chat

In [20]:
digital_marketing_agent = ConversableAgent(
    "digital_marketing_agent",
    system_message="""You are the Digital Marketing Head at a company developing a new B2C tech product. You report to head of marketing.
        Your goal is to communicate with the Marketing head to outline the hiring requirements for assembling a 
        digital marketing team for this project. Clearly and concisely list the key roles, qualifications, and 
        skills, and budget needed for this initiative.""",
    llm_config=config_list,
    human_input_mode="NEVER",  
)


In [21]:
creative_team_agent = ConversableAgent(
    "creative_team_agent",
    system_message="""You are the Creative Team Head at a company developing a new B2C tech product. You report to head of marketing.
        Your goal is to communicate with the Marketing head to outline the hiring requirements for assembling a 
        creative team for this project. Clearly and concisely list the key roles, qualifications, and 
        skills, and budget needed for this initiative.""",
    llm_config=config_list,
    human_input_mode="NEVER",  
)

In [22]:
nested_chats = [
    {
        "recipient": digital_marketing_agent,
        "message": "Gather hiring requirements of your team for this product",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
    {
        "recipient": creative_team_agent,
        "message": "Gather hiring requirements of your team for this product",
        "max_turns": 1,
        "summary_method": "last_msg",
    },
]


In [23]:
marketing_agent.register_nested_chats(
    nested_chats,
    trigger=lambda sender: sender not in [digital_marketing_agent, creative_team_agent],
)

In [24]:
def state_transition(last_speaker, groupchat):
    if last_speaker is hr_agent:
        return product_agent
    elif last_speaker is product_agent:
        return engineering_agent
    elif last_speaker is engineering_agent:
        return marketing_agent
    elif last_speaker is marketing_agent:
        return None

In [25]:
group_chat = GroupChat(
    agents=[product_agent, engineering_agent, marketing_agent],
    messages=[],
    max_round= 10,
    speaker_selection_method= state_transition,
    select_speaker_auto_llm_config=config_list
)

In [26]:
group_chat_manager = GroupChatManager(
    groupchat=group_chat,
)

In [27]:
chat_result = hr_agent.initiate_chat(
    group_chat_manager,
    message="A gaming company focussed on mobiles games is launching a new game designed for PS4. Provide hiring requirements for this new product",
    summary_method="last_msg",
)

hr_agent (to chat_manager):

A gaming company focussed on mobiles games is launching a new game designed for PS4. Provide hiring requirements for this new product

--------------------------------------------------------------------------------

Next speaker: product_agent

product_agent (to chat_manager):

Subject: Hiring Requirements for Product Management Team for New PS4 Game

Dear HR Team,

As we embark on the development of our new mobile game designed for PS4, it is crucial to assemble a dedicated Product Management team that will drive the project from conception to launch. Below are the key roles, qualifications, skills, and budget considerations for this initiative.

### Key Roles and Responsibilities:

1. **Product Manager (2 Positions)**
   - **Qualifications**: 
     - Bachelor’s degree in Business, Marketing, Game Design, or related field.
     - 3-5 years of experience in product management, preferably in the gaming industry.
   - **Skills**:
     - Strong understanding 